# Introduction to Secure Multiparty Computation (SMPC)

This is the first notebook that will explain the basics of multiparty computation. In essence multiparty computation tries to outsource computations to different parties. Now, this per se is easy right? If I want a certain calculation to be performed in a remote computer I just send the data, the function to be calculated and that's it. The problem arises when one wants to outsource a computation without revealing the data, just the function. Secure Multiparty Computation (SMPC) is a set of techniques that allows us to compute a function without explicitely knowing the inputs.


## Yao's Millionaire problem

The typical application example of SMPC is the Yao's millionaire problem: Lets assume there are two millionaires, named Alice and Bob. Alice and Bob what to know who is richer without revealing how much money either one of them have. This is what is known as the millionaires problem, originally introduced by [Andrew Yao](https://en.wikipedia.org/wiki/Andrew_Yao). In essence they just want to know a bit of information, who's richer but not by which ammount. 

In [11]:
from crypt import RSA, PrimesSieveEratosthenes, LCM
from random import seed
#seed(3)

bits = 16
primes = list(PrimesSieveEratosthenes(1<<bits))
N, e, d = RSA(bits, primes)
print(f"(N, e, d) = {N, e, d}")

190724747 352783776
(N, e, d) = (2116794727, 190724747, 28822211)


In [2]:
from crypt import LCM, RandomPrime, BruteForceFactorisation

p = RandomPrime(16, 40)
q = RandomPrime(16, 40)
primes = list(PrimesSieveEratosthenes(1<<16))

print(p, q)

a_factors, _ = BruteForceFactorisation(p-1, primes)
b_factors, _ = BruteForceFactorisation(q-1, primes)

print(a_factors, b_factors)
print(LCM(p-1, q-1, primes))

39191 34721
[2, 5, 3919] [2, 5, 7, 31]
136067680


In [3]:
LCM(21,6)

42